<a href="https://colab.research.google.com/github/HJK-G/AI/blob/master/Colab/CentAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CentAI**

---


By Justin Kim

# DESCRIPTION
This is an AI project to play a game.

It uses Deep Reinforcement Learning (DQNAgent)

https://keon.io/deep-q-learning/

The game is Centakill, made by one of my friends, Kaiway Tang.

Go check them out at: https://kaiway.itch.io/

Or just the game at: https://kaiway.itch.io/centa



NOTE:

This only works if you run this on your local machine.

You can do this by:

1. copying the code into a local Python environment on your computer.
2. connecting this notebook to a local runtime
https://research.google.com/colaboratory/local-runtimes.html

I would recommend option 2 if you want to use notebooks. This allows for easier sharing and it looks cleaner. 

However, option 1 is probably easier.


For Option 1:

1. Install [Python 3.6.8](https://www.python.org/downloads/release/python-368/)

2. Install [PyCharm Community Edition](https://www.jetbrains.com/pycharm/download/)

3. Install [Tesseract OCR](https://github.com/tesseract-ocr/tesseract/wiki)

> For Mac, use Homebrew to install.
>
> For Windows, use [this](https://github.com/UB-Mannheim/tesseract/wiki).

4. Create a new project in PyCharm CE

5. Press the dropdown menu for the Project Interpreter

6. Create a new environment and select the Python that you installed for your base interpreter

7. Go to your settings and find Project Interpreter. You may use the search bar. 

8. Find the plus button and press it to install the packages.

9. Install all of the packages listed below.

10. Create a new Python file

11. Read the notebook and copy the code cells into the file.

12. Change the score_loc and score_dim variables to fit your screen. Currently it is for a 1920 x 1080 resolution.

13. For Windows, change the tesseract_path variable to the location of tesseract.exe (ex: C:\Program Files (x86)\Tesseract-OCR\tesseract.exe)

14. Right click the file and press run.

15. If prompted, give PyCharm access to your entire system. If this doesn't work, run as administrator. 



For Option 2:

This might be for more intermediate level people who want to use Jupyter Notebooks/Google Colab.

1. Download Python https://www.python.org/downloads/

2. Go to your terminal/command line

3. Do ```pip install jupyterlab```

4. Do ```pip install jupyter_http_over_ws```

5. Do ```jupyter serverextension enable --py jupyter_http_over_ws```

6. Do 
```
jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'  --port=8888 --NotebookApp.port_retries=0
```
to start the notebook server. This should open a new tab in your web browser.

7. Go back to this notebook to the top right where it says Connect or RAM and Disk and click on the dropdown menu
![alt text](https://cdn-images-1.medium.com/fit/t/1600/480/1*YWT1HlJeimWw61QYV4vEVA.png)

8. Press connect to local runtime and then press connect

9. If you close out of your terminal, you must do step 6 again to reconnect.

10. Read through this notebook and run each code cell.

11. It may not work the first time, because your terminal doesn't have the correct permissions. If prompted, give your terminal full control of your system (so it can press keys and take screenshots)

12. It should work now. Rerun the entire thing. If you have any questions, post a comment on my Youtube video, and I'll try to help.

#Code

installaton of various packages

run only once at the beginning


In [0]:
!pip install numpy
!pip install keras
!pip install tensorflow
!pip install mss
!pip install Pillow
!pip install opencv-python
!pip install pynput
!pip install pytesseract

If you are doing Option 2, run this cell.

In [0]:
!sudo apt install tesseract-ocr

If you want to use GPU, follow these instructions:

https://www.tensorflow.org/install/gpu

if not, ignore the next cell

In [0]:
!pip uninstall tensorflow
!pip install tensorflow-gpu
import tensorflow as tf
print(tf.test.gpu_device_name())
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

imports:

In [0]:
from collections import deque
import random
import numpy as np
import time

from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.optimizers import Adam

from mss import mss
from PIL import Image
import cv2

from pynput.keyboard import Key, Controller
import pytesseract

This is DQNAgent. It was taken from https://keon.io/deep-q-learning/

This is a class that contains the model.

Tweak the values until the model learns better, these are just the default values.

Every time you change this cell, you have to run the cell. 

In [0]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(batch_size, len(self.memory)))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                         np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self):
        self.model.save_weights('model.h5')

    def load(self):
        self.model.load_weights('model.h5')


Functions for the model to get input and interact with the computer.

Run this cell once.

In [0]:
def capture_ss():
    img = sc.grab(monitor=sc.monitors[1])
    return np.array(Image.frombytes('RGB', img.size, img.bgra, 'raw', 'BGRX'))


def process_img(original_img):
    processed_img = cv2.resize(original_img, scr_dim, interpolation=cv2.INTER_AREA)
    processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
    return processed_img


def shape_img(processed_img):
    shaped_img = np.reshape(processed_img, [1, screen_size])
    return shaped_img


def act(action):
    print(action)
    if action > 4:
        if action < 20:
            if action % 4 == 0:
                keyboard.press('w')
            if action % 4 == 1:
                keyboard.press('a')
            if action % 4 == 2:
                keyboard.press('s')
            if action % 4 == 3:
                keyboard.press('d')
        else:
            if action == 20 or action == 21 or action == 24 or action == 25 or action == 28 or action == 29:
                keyboard.press('w')
            if action == 20 or action == 22 or action == 24 or action == 26 or action == 28 or action == 30:
                keyboard.press('a')
            if action == 23 or action == 22 or action == 27 or action == 26 or action == 31 or action == 30:
                keyboard.press('s')
            if action == 23 or action == 21 or action == 27 or action == 25 or action == 31 or action == 29:
                keyboard.press('d')
    if action == 0 or (8 <= action <= 11) or (20 <= action <= 23):
        keyboard.press('u')
    if action == 1 or (12 <= action <= 15) or (24 <= action <= 27):
        keyboard.press('i')
    if action == 2:
        keyboard.press('o')
    if action == 3 or (16 <= action <= 19) or (28 <= action <= 31):
        keyboard.press('p')


def reset():
    keyboard.release('w')
    keyboard.release('a')
    keyboard.release('s')
    keyboard.release('d')
    keyboard.release('u')
    keyboard.release('i')
    keyboard.release('o')
    keyboard.release('p')

The main function where all the AI happens.

Run this once.

In [0]:
def main():
    agent = DQNAgent(screen_size, 4 + 4 + 12 + 12)
    agent.load()

    death_flag = process_img(capture_ss())[death_loc[1], death_loc[0]]
    print("starting")

    time.sleep(2)
    try:
        # repeat episode_num times
        for e in range(episode_num):
            keyboard.press(Key.tab)
            keyboard.release(Key.tab)

            score = 0
            dead = False
            last_scr = process_img(capture_ss())
            last_scr_vector = shape_img(last_scr)
            action = agent.act(last_scr_vector)
            act(action)
            last_time = time.time()

            while not dead:
                reset()
                screen = capture_ss()
                pro_scr = process_img(screen)
                scr_vector = shape_img(pro_scr)

                score_crop = screen[score_loc[1]:score_loc[1] + score_dim[1],
                             score_loc[0]:score_loc[0] + score_dim[0]]
                score_ocr = pytesseract.image_to_string(score_crop,
                                                        config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
                if score_ocr != '':
                    score = max(score, int(score_ocr))

                death_check = pro_scr[death_loc[1]][death_loc[0]]
                if death_flag == death_check:
                    dead = True

                reward = score

                agent.remember(last_scr_vector, action, reward, scr_vector, dead)

                if dead:
                    print('episode: {}/{}, score: {}'.format(e, episode_num, reward))
                    continue

                print(time.time() - last_time)
                last_time = time.time()
                action = agent.act(scr_vector)
                act(action)
                last_scr = pro_scr
                last_scr_vector = scr_vector

            agent.replay(32)

            if e % 10 == 0:
                agent.save()
    except KeyboardInterrupt:
        pass

    agent.save()

This cell runs the main function. Run as many times as you want.

You may need to change score_loc and score_dim to fit your computer screen. Just use trial and error to figure that out.

episode_num is the number of times it will run. However, if you stop the program by pressing the stop button, the last few episodes will not be saved. 


Before you run this cell, go to the game and make it full screen.

Die first, but don't press tab to restart. 

Come back to this cell, run it, and quickly switch back to the game.

Wait about 5 seconds and the AI should start playing.

In [0]:
# pytesseract.pytesseract.tesseract_cmd = r'full_path_to_your_tesseract_executable'

keyboard = Controller()
sc = mss()

episode_num = 160
scr_dim = (200, 50)
score_loc = (1085, 150)  # x, y
score_dim = (400, 165)  # x, y
death_loc = (int(scr_dim[0] / 2), int(scr_dim[1] / 2))
screen_size = scr_dim[0] * scr_dim[1]

time.sleep(4)
main()


Score location testing code:

Run this to use trial and error to determine the location of your score. 

It should then create a new window where it shows what part of the screen it grabbed. 

Press q to quit that screen and continue to modify the location and dimensions if they are wrong. 

Remember, it should contain all digits of the score, and no rocks or anything.

---

If you are doing this in a local Python environment (Option 1), create a new file and put this code inside that.

If you are doing this on Colab with a local runtime (Option 2), run only this cell.

In [0]:
import time
from mss import mss
import numpy as np
from PIL import Image
import cv2
import pytesseract


def capture_ss():
    img = sc.grab(monitor=sc.monitors[1])
    return np.array(Image.frombytes('RGB', img.size, img.bgra, 'raw', 'BGRX'))


score_loc = (1085, 150)  # x, y
score_dim = (400, 165)  # x, y
# pytesseract.pytesseract.tesseract_cmd = r'full_path_to_your_tesseract_executable'    # remember to change this variable if you are on a local Python environment and on Windows

sc = mss()

time.sleep(4)

screen = capture_ss()

score_crop = screen[score_loc[1]:score_loc[1] + score_dim[1],
             score_loc[0]:score_loc[0] + score_dim[0]]
score_ocr = pytesseract.image_to_string(score_crop,
                                        config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
if score_ocr != '':
    print(int(score_ocr))

while True:
    cv2.imshow('press q to stop', score_crop)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

